In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///olympics.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM medals_total LIMIT 10;")

sqlite
['medals_total']


'[(\'USA\', \'United States\', \'United States of America\', \'40\', \'44\', \'42\', \'126\'), (\'CHN\', \'China\', "People\'s Republic of China", \'40\', \'27\', \'24\', \'91\'), (\'JPN\', \'Japan\', \'Japan\', \'20\', \'12\', \'13\', \'45\'), (\'AUS\', \'Australia\', \'Australia\', \'18\', \'19\', \'16\', \'53\'), (\'FRA\', \'France\', \'France\', \'16\', \'26\', \'22\', \'64\'), (\'NED\', \'Netherlands\', \'Netherlands\', \'15\', \'7\', \'12\', \'34\'), (\'GBR\', \'Great Britain\', \'Great Britain\', \'14\', \'22\', \'29\', \'65\'), (\'KOR\', \'Korea\', \'Republic of Korea\', \'13\', \'9\', \'10\', \'32\'), (\'ITA\', \'Italy\', \'Italy\', \'12\', \'13\', \'15\', \'40\'), (\'GER\', \'Germany\', \'Germany\', \'12\', \'13\', \'8\', \'33\')]'

In [4]:
import os
os.environ["OPENAI_API_KEY"] = os.environ["GITHUB_TOKEN"]


In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})